In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pandas.core.algorithms as algos
import scipy.stats.stats as stats
import re
import string

from pandas import Series

from sklearn.externals import joblib

In [0]:
x_train = joblib.load("/content/drive/My Drive/TFM/data/x_train.pkl").reset_index(drop=True)
y_train = joblib.load("/content/drive/My Drive/TFM/data/y_train.pkl").reset_index(drop=True)
x_test = joblib.load("/content/drive/My Drive/TFM/data/x_test.pkl").reset_index(drop=True)
y_test = joblib.load("/content/drive/My Drive/TFM/data/y_test.pkl").reset_index(drop=True)

**Importante** Este código se basa en el desarrollado [aquí](https://medium.com/@sundarstyles89/weight-of-evidence-and-information-value-using-python-6f05072e83eb).

In [0]:
max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(X, Y, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3 = d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(X, Y):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)


def woe_iv(df1, target):
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        # si la variable es numérica y tiene más de dos números distintos ejecutamos mono_bin
        if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
            conv = mono_bin(df1[i], target)
            conv["VAR_NAME"] = i
            count = count + 1
        else:
            # ejecutamos la variable categórica
            conv = char_bin(df1[i], target)
            conv["VAR_NAME"] = i            
            count = count + 1
        
        # se ejecuta en la primera pasada del bucle. En el resto solo el append
        if count == 0:
            iv_df = conv
        else:
            iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df, iv) 

In [0]:
final_iv, IV = woe_iv(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: RuntimeWarning: divide by zero encountered in log


In [0]:
IV.sort_values('IV', ascending=False)

,VAR_NAME,IV
13,issue_d,0.779984
19,out_prncp,0.704700
12,int_rate,0.470840
25,sub_grade,0.365309
30,total_rec_late_fee,0.229038
22,recoveries,0.157926
3,collection_recovery_fee,0.136263
10,initial_list_status,0.125069
29,total_rec_int,0.115141
11,inq_last_6mths,0.075953


In [0]:
IV.sort_values('IV', ascending=False).T

,13,19,12,25,30,22,3,10,29,11,31,21,15,2,24,32,9,26,1,6,23,8,27,18,7,20,28,16,4,5,17,0,14
VAR_NAME,issue_d,out_prncp,int_rate,sub_grade,total_rec_late_fee,recoveries,collection_recovery_fee,initial_list_status,total_rec_int,inq_last_6mths,total_rec_prncp,purpose,last_pymnt_amnt,annual_inc,revol_util,verification_status,home_ownership,term,addr_state,dti,revol_bal,emp_length,total_acc,open_acc,earliest_cr_line,pub_rec,total_pymnt,last_pymnt_d,collections_12_mths_ex_med,delinq_2yrs,loan_amnt,acc_now_delinq,last_credit_pull_d
IV,0.779984,0.7047,0.47084,0.365309,0.229038,0.157926,0.136263,0.125069,0.115141,0.0759533,0.0547261,0.0516554,0.0510835,0.0467879,0.0341643,0.0330993,0.0215398,0.0195514,0.0166172,0.00881067,0.0075686,0.00575798,0.00509285,0.00271692,0.00242008,0.0022425,0.000392784,0.000360042,0.00016928,0.000153083,5.1371e-05,3.01701e-06,0


In [0]:
final_iv

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,loan_amnt,500,13200,288990,22525,0.077944,266465,0.922056,0.496824,0.500408,-0.007187,0.000051
1,loan_amnt,13225,35000,288844,22813,0.078980,266031,0.921020,0.503176,0.499592,0.007148,0.000051
2,term,0,0,401174,28726,0.071605,372448,0.928395,0.633597,0.699438,-0.098865,0.019551
3,term,1,1,176660,16612,0.094034,160048,0.905966,0.366403,0.300562,0.198081,0.019551
4,int_rate,5.32,7.89,74263,1239,0.016684,73024,0.983316,0.027328,0.137135,-1.613054,0.470840
5,int_rate,7.9,9.25,54270,1646,0.030330,52624,0.969670,0.036305,0.098825,-1.001394,0.470840
6,int_rate,9.32,11.14,67671,3243,0.047923,64428,0.952077,0.071529,0.120992,-0.525620,0.470840
7,int_rate,11.22,12.49,67104,3820,0.056927,63284,0.943073,0.084256,0.118844,-0.343952,0.470840
8,int_rate,12.53,13.53,58206,3809,0.065440,54397,0.934560,0.084013,0.102155,-0.195512,0.470840
9,int_rate,13.55,14.65,64673,5481,0.084749,59192,0.915251,0.120892,0.111160,0.083931,0.470840


In [0]:
transform_vars_list = x_train.columns
transform_prefix = 'new_' # leave this value blank if you need replace the original column values

In [0]:
transform_vars_list

Index(['loan_amnt', 'term', 'int_rate', 'sub_grade', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'total_pymnt',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'last_credit_pull_d', 'collections_12_mths_ex_med', 'acc_now_delinq'],
      dtype='object')

In [0]:
for var in transform_vars_list:
    small_df = final_iv[final_iv['VAR_NAME'] == var]
    transform_dict = dict(zip(small_df.MAX_VALUE,small_df.WOE))
    replace_cmd = ''
    replace_cmd1 = ''
    for i in sorted(transform_dict.items()):
        replace_cmd = replace_cmd + str(i[1]) + str(' if x <= ') + str(i[0]) + ' else '
        replace_cmd1 = replace_cmd1 + str(i[1]) + str(' if x == "') + str(i[0]) + '" else '
    replace_cmd = replace_cmd + '0'
    replace_cmd1 = replace_cmd1 + '0'
    if replace_cmd != '0':
        try:
            x_train[transform_prefix + var] = x_train[var].apply(lambda x: eval(replace_cmd))
        except:
            x_train[transform_prefix + var] = x_train[var].apply(lambda x: eval(replace_cmd1))

In [0]:
x_train.shape

(577834, 66)

In [0]:
x_train.head()

,loan_amnt,term,int_rate,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,acc_now_delinq,new_loan_amnt,new_term,new_int_rate,new_sub_grade,new_emp_length,new_home_ownership,new_annual_inc,new_verification_status,new_issue_d,new_purpose,new_addr_state,new_dti,new_delinq_2yrs,new_earliest_cr_line,new_inq_last_6mths,new_open_acc,new_pub_rec,new_revol_bal,new_revol_util,new_total_acc,new_initial_list_status,new_out_prncp,new_total_pymnt,new_total_rec_prncp,new_total_rec_int,new_total_rec_late_fee,new_recoveries,new_collection_recovery_fee,new_last_pymnt_d,new_last_pymnt_amnt,new_last_credit_pull_d,new_collections_12_mths_ex_med,new_acc_now_delinq
0,16200.0,1,18.49,21,3,RENT,45000.0,Source Verified,102,debt_consolidation,GA,23.44,0.0,752,0.0,25.0,0.0,19531.0,57.6,48.0,1,16033.91,465.63,166.09,299.54,0.0,0.0,0.0,97,482.27,104,0.0,0.0,0.007148,0.198081,0.438427,0.624465,0.072420,0.157232,0.182520,-0.142917,-3.745607,0.041005,-0.067777,0.081159,-0.003300,0.048605,-0.142933,-0.092216,0.006987,-0.037120,0.005520,-0.085151,0.291991,-0.879068,0.019654,0.212343,-0.511105,-0.092336,-0.442034,-0.407297,-0.000107,-0.249256,0,0.000365,0.000028
1,10000.0,0,19.52,18,10,RENT,40000.0,Not Verified,73,credit_card,FL,12.09,0.0,719,1.0,11.0,0.0,7169.0,62.9,14.0,1,0.00,8861.98,5625.67,3236.31,0.0,0.0,0.0,95,370.00,104,0.0,0.0,-0.007187,-0.098865,0.926103,0.548015,-0.079547,0.157232,0.251305,-0.117682,0.638421,-0.323889,0.125943,-0.056147,-0.003300,0.048605,-0.142933,0.033913,0.006987,0.050453,0.073611,0.126868,0.291991,0.855113,-0.019985,-0.258905,0.326594,-0.092336,-0.442034,-0.407297,-0.000107,0.205819,0,0.000365,0.000028
2,8300.0,0,16.99,17,10,RENT,32500.0,Not Verified,96,debt_consolidation,NJ,26.99,0.0,697,1.0,11.0,0.0,14660.0,57.0,19.0,1,6997.14,2063.33,1302.86,760.47,0.0,0.0,0.0,97,295.88,104,0.0,0.0,-0.007187,-0.098865,0.438427,0.478773,-0.079547,0.157232,0.351148,-0.117682,-1.021373,0.041005,0.093156,0.092260,-0.003300,0.048605,-0.142933,0.033913,0.006987,-0.008413,0.005520,0.022132,0.291991,-0.870621,0.019654,0.212343,-0.082412,-0.092336,-0.442034,-0.407297,-0.000107,0.205819,0,0.000365,0.000028
3,12000.0,1,9.99,7,7,OWN,70000.0,Source Verified,95,debt_consolidation,CA,27.14,0.0,370,0.0,11.0,1.0,6026.0,42.7,30.0,0,10723.18,2032.62,1276.82,755.80,0.0,0.0,0.0,97,254.91,104,0.0,0.0,-0.007187,0.198081,-0.525620,-0.374102,-0.079547,-0.060955,-0.073145,-0.142917,-0.731705,0.041005,0.062988,0.092260,-0.003300,-0.049801,-0.142933,0.033913,0.006987,0.050453,-0.141954,-0.058535,-0.432832,-0.870621,0.019654,0.212343,-0.082412,-0.092336,-0.442034,-0.407297,-0.000107,0.205819,0,0.000365,0.000028
4,4000.0,0,9.99,7,10,MORTGAGE,64000.0,Verified,100,major_purchase,NY,27.79,2.0,489,0.0,19.0,0.0,50000.0,27.2,36.0,0,3710.35,382.71,289.65,93.06,0.0,0.0,0.0,97,129.05,104,0.0,0.0,-0.007187,-0.098865,-0.525620,-0.374102,-0.079547,-0.133435,-0.028542,0.251579,-2.396383,0.042522,0.114882,0.092260,0.046392,-0.049801,-0.142933,-0.092216,0.006987,-0.194115,-0.269904,-0.058535,-0.432832,-0.870621,0.019654,0.212343,-0.944616,-0.092336,-0.442034,-0.407297,-0.000107,0.205819,0,0.000365,0.000028


In [0]:
x_train['home_ownership'].value_counts()

MORTGAGE    290321
RENT        233559
OWN          53798
OTHER          125
NONE            29
ANY              2
Name: home_ownership, dtype: int64

In [0]:
x_train['new_home_ownership'].value_counts()

-0.133435    290321
 0.157232    233559
-0.060955     53798
 2.091190       125
 0.303946        29
 0.000000         2
Name: new_home_ownership, dtype: int64

In [0]:
small_df = final_iv[final_iv['VAR_NAME'] == 'home_ownership']

In [0]:
small_df

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
32,home_ownership,ANY,ANY,2,0,0.000000,2,1.000000,0.000000,0.000004,0.000000,0.02154
33,home_ownership,MORTGAGE,MORTGAGE,290321,20131,0.069340,270190,0.930660,0.444020,0.507403,-0.133435,0.02154
34,home_ownership,NONE,NONE,29,3,0.103448,26,0.896552,0.000066,0.000049,0.303946,0.02154
35,home_ownership,OTHER,OTHER,125,51,0.408000,74,0.592000,0.001125,0.000139,2.091190,0.02154
36,home_ownership,OWN,OWN,53798,3990,0.074166,49808,0.925834,0.088006,0.093537,-0.060955,0.02154
37,home_ownership,RENT,RENT,233559,21163,0.090611,212396,0.909389,0.466783,0.398869,0.157232,0.02154
